## Function Definitions, and mock function call results

In [1]:
import json
import uuid
from functools import partial


def add(args: str):
    args = json.loads(args)
    return str(float(args["a"]) + float(args["b"]))


def sub(args: str):
    args = json.loads(args)
    return str(float(args["a"]) - float(args["b"]))


def mult(args: str):
    args = json.loads(args)
    return str(float(args["a"]) * float(args["b"]))


def div(args: str):
    args = json.loads(args)
    return str(float(args["a"]) / float(args["b"]))


def get_oai_response(model, functions, msgs, api_key, base_url):
  import openai
  openai.api_key = api_key 
  openai.base_url = base_url
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      stream=True,
    )
    return completion
  except Exception as e:
    print(e)


def insert_tool_response(res, msgs):
    assistant_message = res.delta
    tool_calls = []
    for tool_call in assistant_message.tool_calls:
        tool_calls.append( {
                            "id": tool_call.id,
                            "function": {"name": tool_call.function.name,
                                        "arguments": tool_call.function.arguments},
                            "type": "function",
                        })
    msgs.append({"role": "assistant",  "tool_calls": tool_calls})
    
    for i, tool_call in enumerate(assistant_message.tool_calls):
        if tool_call.function.name == "getCurrentWeather":
            print()
            l = len((json.loads(assistant_message.tool_calls[i].function.arguments))["location"])
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"temprature is {(i+1) * 50 + l } degree"})
        elif tool_call.function.name == "calculate_distance":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Distance is {(i+1) * 50} miles."})
        elif tool_call.function.name == "generate_password":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Password generated: {uuid.uuid4().hex[:8]}"})
        elif tool_call.function.name == "orderUmbrella":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Order placed. the price is {(i+1) * 10} dollars."})
        elif tool_call.function.name == "list_files":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"File list:\nreport.docx\ntask.txt\nnotes.txt"})
        elif tool_call.function.name == "get_file_size":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"the size is {(i+1) * 100} bytes."})
        elif tool_call.function.name == "addition":
            msgs.append({
                "role": "tool",
                "name": "addition",
                "content": add(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "subtraction":
            msgs.append({
                "role": "tool",
                "name": "subtraction",
                "content": sub(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "multiplication":
            msgs.append({
                "role": "tool",
                "name": "multiplication",
                "content": mult(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "division":
            msgs.append({
                "role": "tool",
                "name": "division",
                "content": div(tool_call.function.arguments),
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "create_3d_model":
            msgs.append({
                "role": "tool",
                "name": "division",
                "content": "created.",
                "tool_call_id": tool_call.id
            })
        print(f"Observation: {msgs[-1]['content']}")
    
    return msgs

def run_completion(chat_method, user_query, msgs=[], model="gpt-4-0125-preview"):
    system_prompt = "You are a helpful assistant."
    functions = [
        {
            "type": "function",
            "function": {
                "name": "list_files",
                "description": "List all files in a directory",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "directory": {
                            "type": "string",
                            "description": "the directory to list files from"
                        }
                    },
                    "required": [
                        "directory"
                    ]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Create a 3D model of an object with specified dimensions",
                "name": "create_3d_model",
                "parameters": {
                    "properties": {
                    "object_name": {
                        "description": "Name of the object to be modeled",
                        "type": "string"
                    },
                    "dimensions": {
                        "description": "Dimensions of the 3D object (length, width, height)",
                        "type": "object",
                        "properties": {
                        "length": {
                            "type": "number"
                        },
                        "width": {
                            "type": "number"
                        },
                        "height": {
                            "type": "number"
                        }
                        },
                        "required": [
                        "length",
                        "width",
                        "height"
                        ]
                    }
                    },
                    "required": [
                    "object_name",
                    "dimensions"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
                "description": "Get the latest insurance premium from a list of premiums.",
                "name": "latest_insurance_premium",
                "parameters": {
                    "properties": {
                        "premiums": {
                            "description": "List of insurance premiums",
                            "type": "array",
                            "items": {
                            "type": "number"
                            }
                        }
                    },
                    "required": [
                        "premiums"
                    ],
                    "type": "object"
                }
            }
        },
        {
            "type": "function",
            "function": {
            "description": "Calculate insurance premium based on age and coverage",
            "name": "calculate_insurance_premium",
            "parameters": {
                "properties": {
                "age": {
                    "description": "Age of the person applying for insurance",
                    "type": "integer"
                },
                "coverage_type": {
                    "description": "Type of insurance coverage",
                    "type": "string",
                    "enum": [
                    "basic",
                    "standard",
                    "premium"
                    ]
                }
                },
                "required": [
                "age",
                "coverage_type"
                ],
                "type": "object"
            }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_file_size",
                "description": "Get the size of a file in bytes",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "filename": {
                            "type": "string",
                            "description": "the name of the file to get its size"
                        }
                    },
                    "required": [
                        "filename"
                    ]
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'addition',
                'description': "Adds two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to add',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to add',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'subtraction',
                'description': "Subtracts two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to be subtracted from',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Number to subtract',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'multiplication',
                'description': "Multiply two numbers together",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to multiply',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to multiply',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
        {
            'type': 'function',
            'function': {
                'name': 'division',
                'description': "Divide two numbers",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'a': {
                            'description': 'First number to use as the dividend',
                            'type': 'string'
                        },
                        'b': {
                            'description': 'Second number to use as the divisor',
                            'type': 'string'
                        }
                    },
                    'required': []
                }
            }
        },
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
      {"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}
       ]
    if not msgs or len(msgs) == 0:
        msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]
    else:
        msgs.append({"role": "user", "content": user_query})

    res = chat_method(model=model, functions=functions, msgs=msgs)
    
    next_content = ""
    l = 0
    print(f"<Turn {l}>:")
    print("[AI response]:")
    for chunk in res:
        if chunk.choices[0].delta.content and len(chunk.choices[0].delta.content) > 0:
            next_content += chunk.choices[0].delta.content
            print(chunk.choices[0].delta.content, end="")
        elif chunk.choices[0].delta.tool_calls:
            # Just for testing rubra tools.cpp purpose, won't work for gpt4.
            res_next = chunk.choices[0]
            print("[AI calling functions]:")
            for tool_call in res_next.delta.tool_calls:
                print(f"Tool Call: {tool_call.function}")
            break
    
    
    while not next_content:
        l += 1
        msgs = insert_tool_response(res_next, msgs)
        print(f"\n<Turn {l}>:")
        res_next = chat_method(model=model, functions=functions, msgs=msgs)
        next_content = ""
        print("[AI response]:")
        for chunk in res_next:
            if chunk.choices[0].delta.content and len(chunk.choices[0].delta.content) > 0:
                next_content += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="")
            elif chunk.choices[0].delta.tool_calls:
                res_next = chunk.choices[0]
                print("[AI calling functions]:")
                for tool_call in res_next.delta.tool_calls:
                    print(f"Tool Call: {tool_call.function}")
                break
        
    msgs.append({"role": "assistant", "content": next_content})
    return msgs
        
    

In [2]:
import openai
local_api_key = "sk-"
local_base_url = "http://localhost:1234/v1/"
get_rubra_response = partial(get_oai_response, api_key=local_api_key, base_url=local_base_url)
model = "Llama-3-8b-function-calling-alpha-v1.gguf"


## Chat

In [3]:
user_query = "how are you?"
msgs = run_completion(get_rubra_response, user_query, model=model)

<Turn 0>:
[AI response]:
I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions or tasks you have! How can I assist you today?

## Multi + Parallel Function Call Test examples

In [4]:
user_query = "What is the distance between San Francisco and Cupertino by driving and by air from both directions?"
msgs = run_completion(get_rubra_response, user_query, model=model)


<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"San Francisco","destination":"Cupertino","mode":"driving"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"Cupertino","destination":"San Francisco","mode":"driving"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 2>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"San Francisco","destination":"Cupertino","mode":"air"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 3>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"Cupertino","destination":"San Francisco","mode":"air"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 4>:
[AI response]:
The distance between San Francisco and Cupertin

In [5]:
user_query1 = "what's the distance between SF and NYC? Use the result value to multiply by 8, and then divide by 2, and then minus 30"
msgs = run_completion(get_rubra_response, user_query1, model=model)


<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"San Francisco","destination":"New York City","mode":"airplane"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"50","b":"8"}', name='multiplication')
Observation: 400.0

<Turn 2>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"400.0","b":"2"}', name='division')
Observation: 200.0

<Turn 3>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"200.0","b":"30"}', name='subtraction')
Observation: 170.0

<Turn 4>:
[AI response]:
The final result after performing the operations is 170.0 miles.

In [6]:
user_query2 = "now order 3 umbrellas for me and generate a password of length 8"
msgs = run_completion(get_rubra_response, user_query2, msgs, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"number_to_buy":3}', name='orderUmbrella')
Observation: Order placed. the price is 10 dollars.

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"length":8}', name='generate_password')
Observation: Password generated: 0cc31a6f

<Turn 2>:
[AI response]:
Your order for 3 umbrellas has been placed, and the price is $10 per umbrella. A random password of length 8 has also been generated: 0cc31a6f.

## Simple Math Chaining

In [7]:
user_query3 = "What is four plus six? What is the result of that plus 2? Take the result and multiply by 5 and then divide by two"


msgs = run_completion(get_rubra_response, user_query3, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"4","b":"6"}', name='addition')
Observation: 10.0

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"10.0","b":"2"}', name='addition')
Observation: 12.0

<Turn 2>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"12.0","b":"5"}', name='multiplication')
Observation: 60.0

<Turn 3>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"a":"60.0","b":"2"}', name='division')
Observation: 30.0

<Turn 4>:
[AI response]:
The result of four plus six is ten. Adding two to that gives twelve. Multiplying twelve by five gives sixty, and dividing sixty by two results in thirty.

# Condition

In [8]:
user_query4 = "check the weather in boston, if it's less than 100 degrees Fahrenheit, calculate the distance from boston to NYC"
msgs = run_completion(get_rubra_response, user_query4, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: temprature is 56 degree

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"origin":"Boston","destination":"NYC","mode":"car"}', name='calculate_distance')
Observation: Distance is 50 miles.

<Turn 2>:
[AI response]:
The temperature in Boston is 56 degrees Fahrenheit, which is below 100 degrees. The distance from Boston to New York City is approximately 50 miles.

In [9]:
user_query5 = "check the weather in boston, if it's greater than 100 degrees Fahrenheit, calculate the distance from boston to NYC"
msgs = run_completion(get_rubra_response, user_query5, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"location":"Boston","unit":"f"}', name='getCurrentWeather')

Observation: temprature is 56 degree

<Turn 1>:
[AI response]:
The temperature in Boston is 56 degrees Fahrenheit, which is not greater than 100 degrees. Therefore, I will not calculate the distance from Boston to NYC as the condition was not met. Is there anything else I can assist you with?

# dependency

In [10]:
user_query6 = "check the size of all files in the 'documents' directory."
msgs = run_completion(get_rubra_response, user_query6, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"directory":"documents"}', name='list_files')
Observation: File list:
report.docx
task.txt
notes.txt

<Turn 1>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"filename":"report.docx"}', name='get_file_size')
Observation: the size is 100 bytes.

<Turn 2>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"filename":"task.txt"}', name='get_file_size')
Observation: the size is 100 bytes.

<Turn 3>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"filename":"notes.txt"}', name='get_file_size')
Observation: the size is 100 bytes.

<Turn 4>:
[AI response]:
The sizes of the files in the 'documents' directory are as follows:
- 'report.docx' is 100 bytes.
- 'task.txt' is 100 bytes.
- 'notes.txt' is 100 bytes.
Is there anything else you would like to know or another task yo

In [13]:
user_query0 = "create a 3d model with length 12, width 21, and height 6"
msgs = run_completion(get_rubra_response, user_query0, model=model)

<Turn 0>:
[AI response]:
[AI calling functions]:
Tool Call: ChoiceDeltaToolCallFunction(arguments='{"object_name":"rectangle","dimensions":{"length":12,"width":21,"height":6}}', name='create_3d_model')
Observation: created.

<Turn 1>:
[AI response]:
The 3D model of the rectangle with dimensions 12x21x6 has been successfully created.